# Data Preparation: San Francisco SFPD Incidents

**Project:** A Tale of Two Cities - Comparative Public Safety Analysis

**Purpose:** This notebook handles data loading, cleaning, preprocessing, and feature engineering for the San Francisco dataset.

**Output:** Clean, analysis-ready dataset saved to `data/processed/sf_incidents_cleaned.csv`

---

## 1. Import Libraries

In [1]:
import pandas as pd
from pathlib import Path

## 2. Data Loading & Initial Inspection

We begin by loading the SFPD incident dataset and examining its structure, dimensions, and key features.

In [2]:
# Load CSV file into a DataFrame
file_path = '../data/raw/sfpd_incidents.csv'
df = pd.read_csv(file_path)

print(f"Dataset loaded: {len(df):,} rows, {len(df.columns)} columns")
print(df.head())

Dataset loaded: 983,556 rows, 29 columns
         Row ID       Incident Datetime Incident Date Incident Time  \
0  150750507041  2025/08/26 11:17:00 PM    2025/08/26         23:17   
1  150752104134  2025/08/27 12:37:00 AM    2025/08/27         00:37   
2  150762309027  2025/07/17 03:00:00 PM    2025/07/17         15:00   
3  150740506244  2025/08/23 09:30:00 PM    2025/08/23         21:30   
4  150723571000  2025/08/15 12:00:00 PM    2025/08/15         12:00   

   Incident Year Incident Day of Week         Report Datetime  Incident ID  \
0           2025              Tuesday  2025/08/26 11:17:00 PM      1507505   
1           2025            Wednesday  2025/08/27 12:37:00 AM      1507521   
2           2025             Thursday  2025/08/27 11:55:00 AM      1507623   
3           2025             Saturday  2025/08/24 02:53:00 PM      1507405   
4           2025               Friday  2025/08/24 07:10:00 PM      1507235   

   Incident Number   CAD Number  ...         CNN Police Distric

In [3]:
# Display dataset info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983556 entries, 0 to 983555
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Row ID                    983556 non-null  int64  
 1   Incident Datetime         983556 non-null  object 
 2   Incident Date             983556 non-null  object 
 3   Incident Time             983556 non-null  object 
 4   Incident Year             983556 non-null  int64  
 5   Incident Day of Week      983556 non-null  object 
 6   Report Datetime           983556 non-null  object 
 7   Incident ID               983556 non-null  int64  
 8   Incident Number           983556 non-null  int64  
 9   CAD Number                762527 non-null  float64
 10  Report Type Code          983556 non-null  object 
 11  Report Type Description   983556 non-null  object 
 12  Filed Online              193890 non-null  object 
 13  Incident Code             983556 non-null  i

## 2. Data Preprocessing

### 2.1 DateTime Processing
The dataset contains separate date and time columns. We'll combine these into a single DateTime index for efficient time-series analysis.

In [4]:
# Convert 'Incident Date' to datetime objects
df['Incident Date'] = pd.to_datetime(df['Incident Date'])

# Combine date and time into a single column
df['Incident DateTime'] = pd.to_datetime(
    df['Incident Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Incident Time']
)

# Set as index
df.set_index('Incident DateTime', inplace=True)

print("DateTime index created successfully")
print(df.head())

DateTime index created successfully
                           Row ID       Incident Datetime Incident Date  \
Incident DateTime                                                         
2025-08-26 23:17:00  150750507041  2025/08/26 11:17:00 PM    2025-08-26   
2025-08-27 00:37:00  150752104134  2025/08/27 12:37:00 AM    2025-08-27   
2025-07-17 15:00:00  150762309027  2025/07/17 03:00:00 PM    2025-07-17   
2025-08-23 21:30:00  150740506244  2025/08/23 09:30:00 PM    2025-08-23   
2025-08-15 12:00:00  150723571000  2025/08/15 12:00:00 PM    2025-08-15   

                    Incident Time  Incident Year Incident Day of Week  \
Incident DateTime                                                       
2025-08-26 23:17:00         23:17           2025              Tuesday   
2025-08-27 00:37:00         00:37           2025            Wednesday   
2025-07-17 15:00:00         15:00           2025             Thursday   
2025-08-23 21:30:00         21:30           2025             Saturday   


In [5]:
# Drop redundant date/time columns
df.drop(['Incident DateTime', 'Incident Date', 'Incident Time'], axis=1, inplace=True, errors='ignore')

print(f"Columns after cleanup: {len(df.columns)}")
print(df.info())

Columns after cleanup: 27
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 983556 entries, 2025-08-26 23:17:00 to 2024-06-01 12:00:00
Data columns (total 27 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Row ID                    983556 non-null  int64  
 1   Incident Datetime         983556 non-null  object 
 2   Incident Year             983556 non-null  int64  
 3   Incident Day of Week      983556 non-null  object 
 4   Report Datetime           983556 non-null  object 
 5   Incident ID               983556 non-null  int64  
 6   Incident Number           983556 non-null  int64  
 7   CAD Number                762527 non-null  float64
 8   Report Type Code          983556 non-null  object 
 9   Report Type Description   983556 non-null  object 
 10  Filed Online              193890 non-null  object 
 11  Incident Code             983556 non-null  int64  
 12  Incident Category         982137 non-nul

### 2.2 Handling Missing Values
Before proceeding with analysis, we need to identify and handle missing data appropriately.

In [6]:
# Calculate missing value percentages
missing_percentage = (df.isnull().sum() / len(df)) * 100

print("Columns with missing values:")
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

Columns with missing values:
Filed Online                80.286837
CAD Number                  22.472437
Supervisor District          5.606493
Analysis Neighborhood        5.573551
Supervisor District 2012     5.553014
Intersection                 5.543965
CNN                          5.543965
Latitude                     5.543965
Longitude                    5.543965
Point                        5.543965
Incident Category            0.144272
Incident Subcategory         0.144272
dtype: float64


In [7]:
# Drop columns with high percentage of missing values (>20%)
columns_to_drop = missing_percentage[missing_percentage > 20].index.tolist()

if columns_to_drop:
    df.drop(columns_to_drop, axis=1, inplace=True)
    print(f"Dropped {len(columns_to_drop)} columns with >20% missing values:")
    print(columns_to_drop)
else:
    print("No columns with >20% missing values")

print(f"\nColumns remaining: {len(df.columns)}")

Dropped 2 columns with >20% missing values:
['CAD Number', 'Filed Online']

Columns remaining: 25


In [8]:
# Drop rows with remaining missing values
rows_before = len(df)
df.dropna(inplace=True)
rows_after = len(df)

print(f"Rows dropped: {rows_before - rows_after:,} ({((rows_before - rows_after)/rows_before)*100:.2f}%)")
print(f"Clean dataset: {rows_after:,} rows")
print(f"Remaining missing values: {df.isnull().sum().sum()}")

Rows dropped: 56,580 (5.75%)
Clean dataset: 926,976 rows
Remaining missing values: 0


## 5. Feature Engineering

Extract temporal features for analysis.

In [9]:
# Create temporal features from the DateTime index
df['Hour'] = df.index.hour
df['Day'] = df.index.day
df['Month'] = df.index.month
df['Year'] = df.index.year
df['Day of Week'] = df.index.dayofweek  # Monday=0, Sunday=6
df['Day of Week Name'] = df.index.day_name()
df['Month Name'] = df.index.month_name()
df['Quarter'] = df.index.quarter
df['Is Weekend'] = df['Day of Week'].isin([5, 6]).astype(int)

print("Temporal features created:")
print(df[['Hour', 'Day of Week Name', 'Month Name', 'Year', 'Is Weekend']].head())

Temporal features created:
                     Hour Day of Week Name Month Name  Year  Is Weekend
Incident DateTime                                                      
2025-08-27 00:37:00     0        Wednesday     August  2025           0
2025-07-17 15:00:00    15         Thursday       July  2025           0
2025-08-23 21:30:00    21         Saturday     August  2025           1
2025-08-15 12:00:00    12           Friday     August  2025           0
2025-08-15 21:45:00    21           Friday     August  2025           0


# 6. Category Harmonization

In [10]:
print(f"\nOriginal unique SF Categories: {df['Incident Category'].nunique()}")

def map_high_level_category(cat):
    cat = str(cat).upper()
    
    # Violent
    if cat in ['ASSAULT', 'ROBBERY', 'HOMICIDE', 'SEX OFFENSE', 'OFFENCES AGAINST THE FAMILY AND CHILDREN', 'WEAPONS OFFENSE', 'WEAPONS CARRYING ETC']:
        return 'Violent'
    
    # Theft/Property
    if cat in ['LARCENY THEFT', 'BURGLARY', 'MOTOR VEHICLE THEFT', 'MALICIOUS MISCHIEF', 'STOLEN PROPERTY', 'ARSON']:
        return 'Theft/Property'
    
    # Disturbance/Suspicious
    if cat in ['SUSPICIOUS OCC', 'DISORDERLY CONDUCT', 'MISSING PERSON', 'DRUG OFFENSE', 'MISCELLANEOUS INVESTIGATION']:
        return 'Disturbance/Suspicious'

    # Traffic/Vehicle
    if cat in ['TRAFFIC VIOLATION ARREST', 'RECOVERED VEHICLE', 'DRIVING UNDER THE INFLUENCE']:
        return 'Traffic/Vehicle'
    
    # Fraud
    if cat == 'FRAUD':
        return 'Fraud'
        
    # Non-Criminal / Admin
    if cat in ['NON-CRIMINAL', 'WARRANT', 'LOST PROPERTY', 'CASE CLOSURE', 'OTHER MISCELLANEOUS', 'OTHER']:
        return 'Non-Criminal/Admin'

    # All others fall into a general 'Other'
    return 'Other'

# Apply the mapping
df['Incident_High_Level_Category'] = df['Incident Category'].apply(map_high_level_category)

print("\nHigh-level categories created:")
print(df['Incident_High_Level_Category'].value_counts())


Original unique SF Categories: 49

High-level categories created:
Incident_High_Level_Category
Theft/Property            443411
Non-Criminal/Admin        172317
Violent                   112849
Disturbance/Suspicious    103055
Traffic/Vehicle            36902
Fraud                      32021
Other                      26421
Name: count, dtype: int64


# 7. Final Schema Synchronization

In [11]:
print("\nStarting final schema synchronization...")
print(f"Columns before cleanup: {len(df.columns)}")

# Define final columns to keep
columns_to_keep = {
    # Key Fields
    'Incident_High_Level_Category': 'Incident_High_Level_Category',
    'Resolution': 'Resolution',
    'Analysis Neighborhood': 'Neighborhood',
    'Police District': 'Police_District',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    
    # Temporal Features
    'Hour': 'Hour',
    'Day': 'Day',
    'Month': 'Month',
    'Year': 'Year',
    'Day of Week': 'Day_of_Week',
    'Day of Week Name': 'Day_of_Week_Name',
    'Month Name': 'Month_Name',
    'Quarter': 'Quarter',
    'Is Weekend': 'Is_Weekend'
}

# Filter DataFrame
df_final = df[columns_to_keep.keys()].copy()

# Rename columns for perfect sync
df_final.rename(columns=columns_to_keep, inplace=True)

print(f"Columns after cleanup: {len(df_final.columns)}")
print("Final column list:")
print(df_final.columns.tolist())


Starting final schema synchronization...
Columns before cleanup: 35
Columns after cleanup: 15
Final column list:
['Incident_High_Level_Category', 'Resolution', 'Neighborhood', 'Police_District', 'Latitude', 'Longitude', 'Hour', 'Day', 'Month', 'Year', 'Day_of_Week', 'Day_of_Week_Name', 'Month_Name', 'Quarter', 'Is_Weekend']


In [12]:
# Drop Final Duplicates ===
print(f"Rows before final duplicate drop: {len(df_final):,}")
duplicates_final = df_final.duplicated().sum()

if duplicates_final > 0:
    print(f"Found {duplicates_final:,} duplicates in df_final. Dropping...")
    df_final.drop_duplicates(inplace=True)
    print(f"New final row count: {len(df_final):,}")
else:
    print("No final duplicates found in df_final.")

Rows before final duplicate drop: 926,976
Found 103,435 duplicates in df_final. Dropping...
New final row count: 823,541


# 8. Data Quality Check

In [13]:
# Final data quality summary
print("=" * 60)
print("FINAL CLEAN DATASET SUMMARY")
print("=" * 60)
print(f"Total Rows: {len(df_final):,}")
print(f"Total Columns: {len(df_final.columns)}")
print(f"Date Range: {df_final.index.min()} to {df_final.index.max()}")
print(f"Missing Values: {df_final.isnull().sum().sum()}")
print(f"Duplicate Rows: {df_final.duplicated().sum()}")
print("\nColumn List:")
print(df_final.columns.tolist())

FINAL CLEAN DATASET SUMMARY
Total Rows: 823,541
Total Columns: 15
Date Range: 2018-01-01 00:00:00 to 2025-11-16 23:50:00
Missing Values: 0
Duplicate Rows: 0

Column List:
['Incident_High_Level_Category', 'Resolution', 'Neighborhood', 'Police_District', 'Latitude', 'Longitude', 'Hour', 'Day', 'Month', 'Year', 'Day_of_Week', 'Day_of_Week_Name', 'Month_Name', 'Quarter', 'Is_Weekend']


# 9. Save Processed Data

Save the clean dataset for use by team members in downstream analysis.

In [14]:
# Create processed data directory if it doesn't exist
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Save to CSV
df_final.to_csv(output_dir / 'sf_incidents_cleaned.csv')

print(f"✅ Clean dataset saved to: {output_dir / 'sf_incidents_cleaned.csv'}")
print(f"File size: {(output_dir / 'sf_incidents_cleaned.csv').stat().st_size / (1024*1024):.2f} MB")

✅ Clean dataset saved to: ../data/processed/sf_incidents_cleaned.csv
File size: 110.39 MB


---

## Note:

**For Team Members:**
- Load this clean dataset using: `pd.read_csv('../data/processed/sf_incidents_cleaned.csv', index_col='Incident DateTime', parse_dates=True)`

